Получаем репозиторий для OID

In [0]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 422, done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 422
Receiving objects: 100% (422/422), 34.08 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (146/146), done.


устанавливаем зависимости

In [0]:
!pip install -r ./OIDv4_ToolKit/requirements.txt

     |████████████████████████████████| 2.5MB 9.5MB/s 
     |████████████████████████████████| 5.8MB 59.6MB/s 
     |████████████████████████████████| 51kB 10.0MB/s 
  Found existing installation: botocore 1.13.32
    Uninstalling botocore-1.13.32:
      Successfully uninstalled botocore-1.13.32
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0


Скачиваем по 10 картинок каждого из 4 классов для обучения (для обучения нужно от 1000, обратите внимание на параметр "limit")

In [0]:
!python ./OIDv4_ToolKit/main.py downloader --classes Tomato Cucumber Strawberry Houseplant --type_csv train --limit 10 --multiclasses 1 -y


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading ['Tomato', 'Cucumber', 'Strawberry', 'Houseplant'] together.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Automatic download.
...145%, 0 MB, 47092 KB/s, 0 seconds pas

Генерируем аннотации 

In [0]:
import pandas as pd
import os

f=pd.read_csv("./OID/csv_folder/train-annotations-bbox.csv")

#For multiple classes use the below, adding as many new LabelNames as needed
#this one is tomato[0] cucumber[1] strawberry[2] plant[3] in that order
numClasses = ['/m/07j87','/m/015x4r','/m/07fbm7','/m/03fp41']
u = f.loc[f['LabelName'].isin(numClasses)]
keep_col = ['LabelName','ImageID','XMin','XMax','YMin','YMax']

new_f = u[keep_col]

new_f['ClassNumber'] = new_f['LabelName']

# adding a new column for Classnumber and setting the values based on LabelName
# so, for this, it's tomato[0] cucumber[1] strawberry[2] plant[3] in that order
new_f.loc[new_f['LabelName'] == '/m/07j87', 'ClassNumber'] = 0
new_f.loc[new_f['LabelName'] == '/m/015x4r', 'ClassNumber'] = 1
new_f.loc[new_f['LabelName'] == '/m/07fbm7', 'ClassNumber'] = 2
new_f.loc[new_f['LabelName'] == '/m/03fp41', 'ClassNumber'] = 3


new_f['width'] = new_f['XMax'] - new_f['XMin']
new_f['height'] = new_f['YMax'] - new_f['YMin']
new_f['x'] = (new_f['XMax'] + new_f['XMin'])/2
new_f['y'] = (new_f['YMax'] + new_f['YMin'])/2
keep_col = ['ClassNumber','ImageID','x','y','width','height']
new_f_2 = new_f[keep_col]

print (new_f_2)

for root, dirs, files in os.walk("./OID/Dataset/train/Tomato_Cucumber_Strawberry_Houseplant/"):  
	for filename in files:

		if filename.endswith(".jpg"):
			fn = filename[:-4]
			nf = new_f_2.loc[new_f_2['ImageID'] == fn]
			keep_col = ['ClassNumber','x','y','width','height']
			new_nf = nf[keep_col]
			print(new_nf)
			imgpath = "./OID/Dataset/train/Tomato_Cucumber_Strawberry_Houseplant/" + fn + ".txt"
			print(imgpath)
			new_nf.to_csv(imgpath, index=False, header=False, sep=' ')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

          ClassNumber           ImageID         x         y     width    height
4616                2  00030b70a3abfcea  0.177187  0.627479  0.143125  0.189802
4617                2  00030b70a3abfcea  0.297812  0.332861  0.150625  0.163362
8027                2  0005817f5c4ef853  0.195000  0.714822  0.383750  0.566604
8352                3  0005bda112291d3b  0.622138  0.635531  0.347329  0.531136
8735                0  0006124f7e35107e  0.427553  0.298251  0.731591  0.525219
...               ...               ...       ...       ...       ...       ...
14609259            3  fff9d0c14a4e5b06  0.920712  0.271429  0.155340  0.120000
14609506            3  fffb6efdc2fea5d9  0.486250  0.652778  0.203750  0.623333
14609507            3  fffb6efdc2fea5d9  0.610000  0.835000  0.107500  0.136666
14609508            3  fffb6efdc2fea5d9  0.850625  0.535556  0.038750  0.066667
14609584            3  fffbe5b9c8a94382  0.421667  0.562500  0.820000  0.810000

[38226 rows x 6 columns]
         Class

Создаем общий файл со списком файлов обучающей выборки

In [0]:
import glob
train_list = glob.glob("./OID/Dataset/train/Tomato_Cucumber_Strawberry_Houseplant/*.jpg")

with open('./OID/train.txt', 'w') as f:
    for item in train_list:
        f.write("%s\n" % item)

Скачиваем по 1000 картинок каждого из 4 классов для валидации

In [0]:
!python ./OIDv4_ToolKit/main.py downloader --classes Tomato Cucumber Strawberry Houseplant --type_csv validation --limit 10 --multiclasses 1 -y


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading ['Tomato', 'Cucumber', 'Strawberry', 'Houseplant'] together.
   [ERROR] | Missing the validation-annotations-bbox.csv file.
[DOWNLOAD] | Automatic download.
...100%, 16 MB, 38440 KB/s, 0 seconds p

Генерируем аннотации

In [0]:
import pandas as pd
import os

f=pd.read_csv("./OID/csv_folder/validation-annotations-bbox.csv")

#For multiple classes use the below, adding as many new LabelNames as needed
#this one is tomato[0] cucumber[1] strawberry[2] plant[3] in that order
numClasses = ['/m/07j87','/m/015x4r','/m/07fbm7','/m/03fp41']
u = f.loc[f['LabelName'].isin(numClasses)]
keep_col = ['LabelName','ImageID','XMin','XMax','YMin','YMax']

new_f = u[keep_col]

new_f['ClassNumber'] = new_f['LabelName']

# adding a new column for Classnumber and setting the values based on LabelName
# so, for this, it's tomato[0] cucumber[1] strawberry[2] plant[3] in that order
new_f.loc[new_f['LabelName'] == '/m/07j87', 'ClassNumber'] = 0
new_f.loc[new_f['LabelName'] == '/m/015x4r', 'ClassNumber'] = 1
new_f.loc[new_f['LabelName'] == '/m/07fbm7', 'ClassNumber'] = 2
new_f.loc[new_f['LabelName'] == '/m/03fp41', 'ClassNumber'] = 3


new_f['width'] = new_f['XMax'] - new_f['XMin']
new_f['height'] = new_f['YMax'] - new_f['YMin']
new_f['x'] = (new_f['XMax'] + new_f['XMin'])/2
new_f['y'] = (new_f['YMax'] + new_f['YMin'])/2
keep_col = ['ClassNumber','ImageID','x','y','width','height']
new_f_2 = new_f[keep_col]

print (new_f_2)

for root, dirs, files in os.walk("./OID/Dataset/validation/Tomato_Cucumber_Strawberry_Houseplant/"):  
	for filename in files:

		if filename.endswith(".jpg"):
			fn = filename[:-4]
			nf = new_f_2.loc[new_f_2['ImageID'] == fn]
			keep_col = ['ClassNumber','x','y','width','height']
			new_nf = nf[keep_col]
			print(new_nf)
			imgpath = "./OID/Dataset/validation/Tomato_Cucumber_Strawberry_Houseplant/" + fn + ".txt"
			print(imgpath)
			new_nf.to_csv(imgpath, index=False, header=False, sep=' ')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

        ClassNumber           ImageID         x         y     width    height
311               1  004a9ec75eca5910  0.500000  0.600373  1.000000  0.799231
1151              3  0148fb9a454a9bd0  0.570058  0.247213  0.707262  0.494425
1152              3  0148fb9a454a9bd0  0.559303  0.696292  0.265697  0.400369
1227              3  016b7e5c231d8ac5  0.108648  0.672592  0.217297  0.384997
1228              3  016b7e5c231d8ac5  0.505803  0.273745  0.846163  0.387242
...             ...               ...       ...       ...       ...       ...
203133            2  fe1926390ed1eb71  0.415799  0.338815  0.231747  0.232004
203549            3  fe908de238143eb4  0.500115  0.487253  0.999325  0.973602
203784            3  fee5b4ab59c4a6aa  0.500047  0.500000  0.999905  1.000000
203964            3  ff12f9b36f610386  0.499484  0.499188  0.998967  0.997622
204616            3  ffff21932da3ed01  0.444043  0.495436  0.532506  0.166268

[961 rows x 6 columns]
        ClassNumber         x         y 

Создаем общий файл со списком файлов валидационной выборки

In [0]:
import glob
train_list = glob.glob("./OID/Dataset/validation/Tomato_Cucumber_Strawberry_Houseplant/*.jpg")

with open('./OID/valid.txt', 'w') as f:
    for item in train_list:
        f.write("%s\n" % item)

Создадим файл с именами классов

In [0]:
%%writefile ./OID/obj.names
tomato
cucumber
strawberry
plant

Writing ./OID/obj.names


In [0]:
!cat ./OID/obj.names

tomato
cucumber
strawberry
plant

Создадим файл с путями до изображений датасета

In [0]:
%%writefile ./OID/obj.data
classes = 4
train = ./OID/train.txt
valid = ./OID/valid.txt
names = ./OID/obj.names
backup = ./OID

Writing ./OID/obj.data


In [0]:
!cat ./OID/obj.data

classes = 4
train = ./OID/train.txt
valid = ./OID/valid.txt
names = ./OID/obj.names
backup = ./OID

Скачиваем даркнет

In [0]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 11677 (delta 6), reused 15 (delta 6), pack-reused 11656
Receiving objects: 100% (11677/11677), 10.99 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (7908/7908), done.


Компилируем даркнет

In [0]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile        #С поддержкой вычислений на видеокарте
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile  #С поддержкой OpenCV
!cat Makefile
!make
%cd ..

/content/darknet
GPU=1
CUDNN=0
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)

DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	  -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp

Скачиваем веса для YOLOV3_TINY и извлекаем нужный слой

In [0]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights
!./darknet/darknet partial ./darknet/cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

--2019-12-11 14:05:30--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  13.6MB/s    in 2.5s    

2019-12-11 14:05:33 (13.6 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]

net.optimized_memory = 0 
batch = 1, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 

Обновляем конфигурационный файл для нужного количества классов

In [0]:
!sed -i 's/subdivisions=2/subdivisions=8/g' ./darknet/cfg/yolov3-tiny_obj.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/g' ./darknet/cfg/yolov3-tiny_obj.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/g' ./darknet/cfg/yolov3-tiny_obj.cfg
!sed -i 's/classes=80/classes=4/g' ./darknet/cfg/yolov3-tiny_obj.cfg
!sed -i 's/filters=255/filters=27/g' ./darknet/cfg/yolov3-tiny_obj.cfg
!cat ./darknet/cfg/yolov3-tiny_obj.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=8
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 6000
policy=steps
steps=4800,5400
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=1

[convolutional]
batch_normalize=1
filters=1024
s

Запускаем обучение сети

In [0]:
### TRAIN TINY YOLOv3
!./darknet/darknet detector train ./OID/obj.data ./darknet/cfg/yolov3-tiny_obj.cfg yolov3-tiny.conv.15 -dont_show

yolov3-tiny_obj
net.optimized_memory = 0 
batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max                2x 2/ 2     26 x  26 x 256 ->   13 x  13 x 256 0.000 BF
  10 conv    512       3 x 3/ 1     13 x  13 x 256 -